# 所有张量的操作可以分为两个部分，一个是张量之间的操作，一个是内部的改变，所以我们可以定义两个约束语言，这样代码变得很好维护。
这里指的就是einsum和 rearrage

下面这段代码实现了对于每一张图片每一个像素点的线性映射，读起来不好理解

In [ ]:
import torch

channels_last = torch.randn(64, 32, 32, 3) # (batch, height, width, channel)

B = torch.randn(32*32, 32*32)

## Rearrange an image tensor for mixing across all pixels

channels_last_flat = channels_last.view(
-1, channels_last.size(1) * channels_last.size(2), channels_last.size(3)
)

channels_first_flat = channels_last_flat.transpose(1, 2)
channels_first_flat_transformed = channels_first_flat @ B.T
channels_last_flat_transformed = channels_first_flat_transformed.transpose(1, 2)
channels_last_transformed = channels_last_flat_transformed.view(*channels_last.shape)

但是使用规约语言就十分的顺畅

In [17]:
from einops import einsum, rearrange
import torch
height = width = 32

channels_last = torch.randn(64, height, width, 3)
B = torch.randn(32*32, 64*64)
channels_first = rearrange(
    channels_last,
    "batch height width channel -> batch channel height width"
)
print(f"channels_first.shape: {channels_first.shape}")
channels_first_flat = rearrange(
    channels_first,
    "batch channel height width -> batch channel (height width)"
)
print(channels_first_flat.shape)

channels_first_flat_transformed = einsum(
    channels_first_flat, B,
    "batch  channel hin, hin hout  -> batch channel hout"
)
print(channels_first_flat_transformed.shape)

channels_first.shape: torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 1024])
torch.Size([64, 3, 4096])


In [20]:
import torch 

a =torch.ones(1,10)
print(a)
torch.nn.init.trunc_normal_(a,mean = 0, std = 0.1, a =-0.05, b =0.05)
print(a)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[-0.0183,  0.0090, -0.0021, -0.0199, -0.0494, -0.0481, -0.0379,  0.0444,
         -0.0296, -0.0204]])
